<a href="https://colab.research.google.com/github/Math-an/Genai_image_generation/blob/main/imagegeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'uvicorn[standard]' fastapi websockets diffusers transformers pyngrok pillow wsproto

import json
import torch
import asyncio
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from io import BytesIO
import base64
from PIL import Image
from diffusers import StableDiffusionPipeline
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn


device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "stabilityai/stable-diffusion-2-1"
pipeline = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
).to(device)

app = FastAPI()

def generate_image(prompt: str, num_inference_steps: int, guidance_scale: float):
    try:

        image = pipeline(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]


        buffered = BytesIO()
        quality = 85
        while True:
            buffered.seek(0)
            image.save(buffered, format="JPEG", quality=quality)
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            if len(img_str.encode('utf-8')) < 500 * 1024 or quality <= 10:
                break
            quality -= 5
        return img_str
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            # Receive and process the prompt data
            data = await websocket.receive_text()
            prompt_data = json.loads(data)
            prompt = prompt_data["prompt"]
            num_images = prompt_data.get("num_images", 1)
            quality = prompt_data.get("quality", 35)
            accuracy = prompt_data.get("accuracy", 4.5)


            for _ in range(num_images):
                img_str = generate_image(prompt, quality, accuracy)
                if img_str:
                    await websocket.send_text(img_str)
                else:
                    await websocket.send_text("Error generating image.")
    except WebSocketDisconnect:
        print("WebSocket disconnected.")
    except asyncio.CancelledError:
        print("WebSocket connection closed or canceled.")
    finally:
        if websocket.close_code is None:
            await websocket.close()


nest_asyncio.apply()

ngrok_authtoken = "2oQouIBfPxP3fWYNWHPnZscxQU3_5yuV9f146NWRB4fDi4yR7"  # Replace with your Ngrok authtoken
conf.get_default().auth_token = ngrok_authtoken
public_url = ngrok.connect(8000, "http")  # Use "http" protocol to support WebSockets


websocket_url = public_url.public_url.replace("http", "ws")
print("Public WebSocket URL:", websocket_url)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

...

  0%|          | 0/48 [00:00<?, ?it/s]

INFO:     ('2401:4900:4dd4:2b:d561:6918:a595:6059', 0) - "WebSocket /ws" [accepted]
INFO:     ('2401:4900:4dd4:2b:d561:6918:a595:6059', 0) - "WebSocket /ws" [accepted]
INFO:     ('2401:4900:4dd4:2b:d561:6918:a595:6059', 0) - "WebSocket /ws" [accepted]
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/websockets/websockets_impl.py", line 242, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette

WebSocket disconnected.
WebSocket disconnected.
WebSocket disconnected.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/websockets/websockets_impl.py", line 242, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 152, in __call__
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/exceptions.py", line 62, in 

WebSocket disconnected.
